In [ ]:
import re
import os
import string
import json
import pandas as pd
import pandas_profiling

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

# Verify TXT Describe Table

In [ ]:
f = open(dir_csv+'maint_order'+".txt", "r",encoding='utf-8')
tt = f.read()
tt = tt.replace("\t", " ")
tt = tt.replace("\n", " ")
tt = tt.split('{')

k = []
for i in tt:
    y = []
    for t in i.split(','):
        u = t.strip()
        if any(c in u for c in ['desc','name','type','useNull']):
            try:
                u1 = u.split(':')
                u2 = u1[1].strip()
                if len(re.findall(r"(?<=\')(.*?)(?=\')",u2)) > 0:
                    u3 = re.findall(r"(?<=\')(.*?)(?=\')",u2)
                if len(re.findall(r'(?<=\")(.*?)(?=\")',u2)) > 0:
                    u3 = re.findall(r'(?<=\")(.*?)(?=\")',u2)
                if u2[:5].strip() in ['true','false']:
                    u3 = [ u2[:5].strip() ] 
                else:
                    pass
                y.append(u3[0])
            except:
                pass
    if len(y) == 4:
        k.append(y)
    else:
        pass

data_extrac = pd.DataFrame(k)
data_extrac = data_extrac.rename(columns={data_extrac.columns[0]: 'desc',data_extrac.columns[1]: 'col',data_extrac.columns[2]: 'type', data_extrac.columns[3]: 'usenull'})
data_extrac['usenull'] = data_extrac['usenull'].astype(bool)
data_extrac['usenull'] = data_extrac['usenull'].astype(int)
data_extrac = data_extrac[:126]

In [ ]:
with open('./config/tipocampopeso.json') as f:
    dict_type = json.load(f)

df_type = pd.DataFrame.from_dict(dict_type['type'],orient='index').reset_index()
df_type = df_type.rename(columns={'index':'type',0:'value'})

In [ ]:
df_merge = pd.merge(data_extrac,df_type,on=['type'],how='left')
df_merge = df_merge.rename(columns={'value':'typevalue'})

In [ ]:
pathFile = './base_manusis/evaluation/maint_order.csv'
df_eval = pd.read_csv(pathFile, delimiter=';')
df_merge = df_merge[df_merge.col.isin(df_eval.columns)]
df_merge = df_merge[['col','typevalue','usenull']]

In [ ]:
# Check columns
# list(set(df_merge.name.unique()) - set(df_eval.columns))

# Base Analysis

In [ ]:
dir_csv = "./base_manusis/original/"
dir_save = "./base_manusis/preprocess_new/"
table = "maint_order"

data = pd.read_csv(dir_csv+table+".csv",low_memory=False)

# profile = pandas_profiling.ProfileReport(data)
# profile.to_file(output_file=dir_csv+table+"_describe.html")

In [ ]:
data['opened_at'] = pd.to_datetime(data['opened_at'], errors = 'coerce')
start_date = '01-01-2017'
end_date = '12-31-2019'
mask = (data['opened_at'] > start_date) & (data['opened_at'] <= end_date)
data = data.loc[mask]

data['description'] = data['description'].str.lower()
data['user_text'] = data['user_text'].str.lower()

print(data.shape)

In [ ]:
def counterize(df, columns):
    """
    Extratify data missing values, diverity in values and count rows of max class
    """
    df = df[columns]
    
    info = [['col','qt_missing','qt_diversity','qt_max_row_class','p_null','p_unique','p_max_row_class']]
    
    tot_rows, _ = data.shape
    
    
    for col in columns:
        try:
            # Qtde Valores Nulos
            missing = df[col].isna().sum()
            
            # Qtde de classes diferentes
            diversity = len(df[col].unique()) 

            # Qtde de registros para maior classe
            max_row_class = df.groupby([col]).size().reset_index(name='count').sort_values(by=['count'],ascending=False)['count'].max()
             
            info.append([col, missing, diversity, max_row_class, missing/tot_rows, diversity/tot_rows, max_row_class/tot_rows])
        except:
            pass

    
    df_aux = pd.DataFrame(info[1:],columns=info[0])
    
    return df_aux

In [ ]:
df_analysis = counterize(data,data.columns)

In [ ]:
df_analysis.head()

In [ ]:
df_result = pd.merge(df_analysis,df_merge,on='col',how='left').dropna()

In [ ]:
df_result = df_result[['col','typevalue','usenull','p_null','p_unique','p_max_row_class']]

# Result to Imput in TOPSIS

In [ ]:
df_result.to_csv('./resource/imput_topsis.csv',index=False)

# Export data to ML

In [ ]:
data = data[[*list(df_result.col.values)]]

In [ ]:
data.to_csv('./resource/maint_order_process.csv',index=False)

In [ ]:
data.shape

# Stop CODE

# Others

In [ ]:
# df_analysis.to_csv('./base_manusis/analysis/'+table+'.csv',sep=',',index=False)

In [ ]:
# df_analysis.sort_values(by=['col'])

In [ ]:
# Select columns based in condition

# df_select = df_analysis.query("pr_missing<0.5 and pr_diversity <0.5 and pr_max_row_class<0.8").sort_values(by=['pr_max_row_class'],ascending=True)

In [ ]:
select_cols = df_select.col.values

In [ ]:
len(list(select_cols))

In [ ]:
df_table_full = data[list(select_cols)]

list_exclude = ['cancel_user_id','origin_doc','prog_year_week_number','maint_order_cancel_reason_id',
                'total_cost','rpn','what','when','who','why','how','maint_order_on_hold_reason_id','performed_worktime',
                'estimated_worktime','performed_asset_downtime','id_mobile','mobile_uuid','rework_cos','index',
                'priority_calculated','maint_req_priority','maint_req_id',
                'user_text','scheduled_to_origin','est_finish_at','scheduled_to','est_finish_at_origin','logged_user_id']

for i in list_exclude:
    try:
        df_table_full = df_table_full.drop(i, axis=1)
    except:
        pass

In [ ]:
df_table_full.head()

In [ ]:
list(df_table_full.columns)

In [ ]:
# df_table_full.groupby(['priority','maint_order_status_id']).count()

In [ ]:
# df_table_full.to_csv(dir_save+table+"_etl.csv", index=False)

In [ ]:
df_table_full.shape

# Old

In [ ]:
# Tipos de campos a desconsiderar
typeNotConsider = ['date','datetime','string','text']
# Tabela para processar
table = "maint_order"
# maint_req , maint_order
#============================================

dir_csv = "./base_manusis/original/"
dir_save = "./base_manusis/preprocess/"

data = pd.read_csv(dir_csv+table+".csv",low_memory=False)

data['opened_at'] = pd.to_datetime(data['opened_at'], errors = 'coerce')
start_date = '01-01-2017'
end_date = '12-31-2019'
mask = (data['opened_at'] > start_date) & (data['opened_at'] <= end_date)
data = data.loc[mask]

In [ ]:
# data_describe = data_extrac[['desc','type']].groupby(['type']).count()
# columns = data_extrac['name'].tolist()

# data_extrac = data_extrac[data_extrac['name'].isin(list(data.columns.values))]
# select_columns = data_extrac[~data_extrac['type'].isin(typeNotConsider)]
# select_columns = select_columns['name'].tolist()

# data_select = data[select_columns].fillna(0)

# list_columns = set(data_select.columns)
# list_columns_select = []

# for x in list_columns:
#     data_select[x] = data_select[x].astype('category')
#     if len(data_select[x].dtype.categories) > 1 and any(data_select[x].duplicated()):
#         list_columns_select.append(x)
# data_select = data_select[list_columns_select]

# data_select = data_select.drop('estimated_worktime', axis=1)
# data_select = data_select.drop('performed_asset_downtime', axis=1)
# data_select = data_select.drop('performed_worktime', axis=1)
# data_select = data_select.drop('priority_calculated', axis=1)
# data_select = data_select.drop('maint_req_priority', axis=1)
# data_select = data_select.drop('maint_req_id', axis=1)

# list_exclude = ['cancel_user_id','origin_doc','prog_year_week_number','maint_order_cancel_reason_id',
#                 'total_cost','rpn','what','when','who','why','how','maint_order_on_hold_reason_id','performed_worktime',
#                 'estimated_worktime','performed_asset_downtime','id_mobile','mobile_uuid','rework_cos','index',
#                 'priority_calculated','maint_req_priority','maint_req_id']

# for i in list_exclude:
#     try:
#         data_select = data_select.drop(i, axis=1)
#     except:
#         pass
    
# data_select.to_csv(dir_save+table+"_etl.csv", index=False)
# profile = pandas_profiling.ProfileReport(data_select)
# profile.to_file(outputfile=dir_save+table+"_describe.html")

In [ ]:
# df_table_full.groupby(['user_text']).size()
# df_table_full[df_table_full['user_text'].str.contains("testes")]

In [ ]:
# for column in df_table_full.columns:
#     print(column)
#     display(df_table_full.groupby([column]).size().reset_index(name='count').sort_values(by=['count'],ascending=False))

In [ ]:
def diversity_percentage(df, columns):
    """
    This function returns the number of different elements in each column as a percentage of the total elements in the group.
    A low value indicates there are many repeated elements.
    Example 1: a value of 0 indicates all values are the same.
    Example 2: a value of 100 indicates all values are different.
    """
    diversity = dict()

    for col in columns:
        diversity[col] = len(df[col].unique())

    diversity_series = pd.Series(diversity)
    return (diversity_series/len(df)).sort_values()

In [ ]:
diversity_percentage(df_table_full,df_table_full.columns)

In [ ]:
percent_missing = data.isna().mean()
total_missing = data.isna().sum()

missing_value_df = pd.DataFrame({'column_name': data.columns,
                                 'total_missing': total_missing,
                                 'percent_missing': percent_missing})
missing_value_df = missing_value_df.reset_index();
missing_value_df = missing_value_df.drop('index',axis=1)